# Computation on NumPy Arrays: Universal Functions

Computation on NumPy arrays can be very fast, or it can be very slow

The key to making it fast is to use *vectorized* operations, generally implemented through NumPy's __universal functions (ufuncs).__

In [2]:
import numpy as np
for i in range(5):
    print("something")
print()


something
something
something
something
something



# The Slowness of Loops

Python is a dynamic and interpreted language meaning sequences of operations cannot be compiled down to efficient machine code.

Slugishness manifests itself in situations where many small operations are being repeated

- e.g looping over arrays to operate on each element.

In [3]:
import numpy as np
np.random.seed(10)   # seeding the random number to fix value ie. seed 1 have [1,2,3,4,5] and seed 2 have [6,8,1,2,5]

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
print(values)
compute_reciprocals(values)

[5 1 2 1 2]


array([0.2, 1. , 0.5, 1. , 0.5])

In [4]:
big_array = np.random.randint(1, 100, size=100000)
%timeit compute_reciprocals(big_array)

153 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Seems slow for 1 million operations...

Type checking and function dispatches is the culprit.

If we were working with compiled code the type wouldn't have to be checked as rigorously for each item, so computation could be more efficient...

In [5]:
%timeit?

Docstring:
Time execution of a Python statement or expression

Usage, in line mode:
  %timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] statement
or in cell mode:
  %%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] setup_code
  code
  code...

Time execution of a Python statement or expression using the timeit
module.  This function can be used both as a line and cell magic:

- In line mode you can time a single-line statement (though multiple
  ones can be chained with using semicolons).

- In cell mode, the statement in the first line is used as setup code
  (executed but not timed) and the body of the cell is timed.  The cell
  body has access to any variables created in the setup code.

Options:
-n<N>: execute the given statement <N> times in a loop. If <N> is not
provided, <N> is determined so as to get sufficient accuracy.

-r<R>: number of repeats <R>, each consisting of <N> loops, and take the
best result.
Default: 7

-t: use time.time to measure the time, which is the default on Unix.
This f

# Introducing UFuncs

NumPy provides a convenient interface to statically typed, compiled routines

- Known as *vectorized* operation
- Operation applied to the array, which in turn is applied to *each element*.
- Pushes loop into compiled layer underlying NumPy, making execution faster.

In [6]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.2 1.  0.5 1.  0.5]
[0.2 1.  0.5 1.  0.5]


In [7]:
%timeit (1.0 / big_array)

124 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
# ufuncs can operate between two arrays
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

In [9]:
x = np.arange(9).reshape((3, 3))
# ufuncs can be applied to multidimensional arrays
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]], dtype=int32)

# Exploring NumPy's UFuncs

Ufuncs exist in two flavors: 
- unary ufuncs: which operate on a single input
- binary ufuncs: which operate on two inputs.

## Array arithmetic

Feel quite natural as they all use standard arithmetic:

In [10]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


In [11]:
print("-x     = ", -x) #negation
print("x ** 2 = ", x ** 2) # 
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


In [12]:
# Operators can also be combined
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

In [13]:
y = np.arange(1000).reshape(10,100)
print(y.ndim)
y


2


array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99],
       [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
        139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
        152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
        165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 17

In [14]:
y = np.arange(250).reshape(25,10,1)    #    Multiply all 3 variable should = arange size
y

array([[[  0],
        [  1],
        [  2],
        [  3],
        [  4],
        [  5],
        [  6],
        [  7],
        [  8],
        [  9]],

       [[ 10],
        [ 11],
        [ 12],
        [ 13],
        [ 14],
        [ 15],
        [ 16],
        [ 17],
        [ 18],
        [ 19]],

       [[ 20],
        [ 21],
        [ 22],
        [ 23],
        [ 24],
        [ 25],
        [ 26],
        [ 27],
        [ 28],
        [ 29]],

       [[ 30],
        [ 31],
        [ 32],
        [ 33],
        [ 34],
        [ 35],
        [ 36],
        [ 37],
        [ 38],
        [ 39]],

       [[ 40],
        [ 41],
        [ 42],
        [ 43],
        [ 44],
        [ 45],
        [ 46],
        [ 47],
        [ 48],
        [ 49]],

       [[ 50],
        [ 51],
        [ 52],
        [ 53],
        [ 54],
        [ 55],
        [ 56],
        [ 57],
        [ 58],
        [ 59]],

       [[ 60],
        [ 61],
        [ 62],
        [ 63],
        [ 64],
        [ 65]

In [15]:
%timeit y ** 2

633 ns ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Each of the previous operation are *wrappers* for specific NumPy functions:

- e.g `+` is the wrapper for the `add` function. 

In [16]:
print(np.add(x, 2))
print()
print(x + 2)

[2 3 4 5]

[2 3 4 5]


## Absolute value

NumPy also interacts with other inbuilt Python arithmetic operators: 

- e.g Python's built-in absolute value function

In [17]:
x = np.array([-2, -1, 0, 1, 2])
%timeit abs(x)

317 ns ± 17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [18]:
%timeit np.abs(x)


337 ns ± 20.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Trigonometric functions

NumPy provides a large number of useful ufuncs, and some of the most useful for the data scientist are the trigonometric functions.

In [19]:
theta = np.linspace(0, np.pi, 3)

In [20]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


In [21]:
np.linspace?

Signature:
np.linspace(
    start,
    stop,
    num=50,
    endpoint=True,
    retstep=False,
    dtype=None,
    axis=0,
)
Docstring:
Return evenly spaced numbers over a specified interval.

Returns `num` evenly spaced samples, calculated over the
interval [`start`, `stop`].

The endpoint of the interval can optionally be excluded.

.. versionchanged:: 1.16.0
    Non-scalar `start` and `stop` are now supported.

.. versionchanged:: 1.20.0
    Values are rounded towards ``-inf`` instead of ``0`` when an
    integer ``dtype`` is specified. The old behavior can
    still be obtained with ``np.linspace(start, stop, num).astype(int)``

Parameters
----------
start : array_like
    The starting value of the sequence.
stop : array_like
    The end value of the sequence, unless `endpoint` is set to False.
    In that case, the sequence consists of all but the last of ``num + 1``
    evenly spaced samples, so that `stop` is excluded.  Note that the step
    size changes when `endpoint` is Fals

In [22]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


## Exponents and logarithms

Another common type of operation available in a NumPy ufunc are the exponentials:

In [23]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


In [24]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


## Advanced ufunc features

Specifying output

In [4]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)   # Convert automatically to float
print(y)

z = np.multiply(x, 10)
print(z)

[ 0. 10. 20. 30. 40.]
[ 0 10 20 30 40]


In [7]:
import numpy as np
y = np.zeros(10)
print(y)
np.power(2, x, out=y[::2])  # every two space 
print(y)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


## Aggregates

For binary ufuncs, there are some interesting aggregates that can be computed directly from the object.

- e.g `reduce` applies a given operation to the elements of an array til a single result remains

In [27]:
x = np.arange(1, 6)
print(np.add.reduce(x))
print(x)

15
[1 2 3 4 5]


In [28]:
np.multiply.reduce(x)

120

In [29]:
# If we'd like to store all the intermediate results of the computation, we can instead use accumulate
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15], dtype=int32)

In [30]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120], dtype=int32)

## Outer products

Finally, any ufunc can compute the output of all pairs of two different inputs using the `outer` method.

In [31]:
x = np.arange(1, 6)
np.multiply.outer(x, x)
# consider the first row and first column

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

# Summary: ufuncs:

- Help speed up computation significantly

- Useful for array arithmetic, applying operations to all values.

- Also useful for aggregate functions

- __N.B__ if you're stuck with this stuff don't forget the inbuilt help `?` after a function.

In [32]:
np.multiply.outer?

Docstring:
outer(A, B, /, **kwargs)

Apply the ufunc `op` to all pairs (a, b) with a in `A` and b in `B`.

Let ``M = A.ndim``, ``N = B.ndim``. Then the result, `C`, of
``op.outer(A, B)`` is an array of dimension M + N such that:

.. math:: C[i_0, ..., i_{M-1}, j_0, ..., j_{N-1}] =
   op(A[i_0, ..., i_{M-1}], B[j_0, ..., j_{N-1}])

For `A` and `B` one-dimensional, this is equivalent to::

  r = empty(len(A),len(B))
  for i in range(len(A)):
      for j in range(len(B)):
          r[i,j] = op(A[i], B[j])  # op = ufunc in question

Parameters
----------
A : array_like
    First array
B : array_like
    Second array
kwargs : any
    Arguments to pass on to the ufunc. Typically `dtype` or `out`.
    See `ufunc` for a comprehensive overview of all available arguments.

Returns
-------
r : ndarray
    Output array

See Also
--------
numpy.outer : A less powerful version of ``np.multiply.outer``
              that `ravel`\ s all inputs to 1D. This exists
              primarily for compatibili